In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 11.9 MB/s eta 0:00:00


In [2]:
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_histogram

import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('light_background') # optional

In [3]:
import numpy as np

In [4]:
from qiskit.circuit import Parameter

theta = Parameter('θ')

In [5]:
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 94.7 MB/s eta 0:00:00


In [6]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile


In [7]:
pip install pylatexenc --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=72043ad5289b46760de4cf2243f846bd117b6dec3757d5885f72e46c2419a5d6
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc


In [9]:
def get_theta(n):
  return (n+1) * np.pi/3

In [10]:
def build_circuit(params1_vals, params2_vals, grid):
  dim = 4
  num_qubits = 4
  # create a 20x20 nested list for the snp genotypes
  qc = QuantumCircuit(dim)

  for i in range(dim):
    for j in range(dim):
      theta = get_theta(grid[i][j])
      if j == 0:
        qc.h(i)
      if grid[i][j] == -1:
        qc.id(i)
      elif j % 2 == 0:
        qc.rx(theta, i)
      elif j % 2 == 1:
        qc.rz(theta, i)

  for qubit in range(num_qubits):
      qc.ry(params1_vals[qubit], qubit)

  qc.cx(3, 0)

  for qubit in range(num_qubits - 1):
      qc.cx(qubit, qubit + 1)

  for qubit in range(num_qubits):
      qc.ry(params2_vals[qubit], qubit)

  for qubit in reversed(range(num_qubits)):
      qc.cx(qubit - 1, qubit)
  qc.measure_all()
  return qc

In [11]:
from scipy.optimize import minimize

In [12]:
from qiskit.quantum_info import Statevector


In [15]:
statevectors_list = []

In [16]:
# Use the statevector simulator backend for this demonstration.
def cost_function(x):
    num_qubits = 4
    """
    Cost function for the variational algorithm.
    x is a vector of length 40 (first 20 for params1, next 20 for params2).

    For demonstration, we simulate the circuit and compute the distance
    between the resulting statevector and a target state.
    (In your application, you would design a cost function that
     encourages the circuit to output feature vectors that improve your ML task.)
    """

    # Split parameters: first half for first RY layer, second half for second RY layer.
    params1_vals = x[:num_qubits]
    params2_vals = x[num_qubits:]

    # Build the circuit with these parameter values.
    circuit = build_circuit(params1_vals, params2_vals, grid)

    # Execute the circuit to obtain the statevector.
    #job = execute(circuit, backend)

    qc_aer = transpile(circuit, backend=AerSimulator())
    simulator_aer = AerSimulator()
    results = simulator_aer.run(qc_aer, shots=1024).result()
    counts=results.get_counts()
    #statevector = results.get_statevector(circuit)
    circuit.remove_final_measurements()
    statevector = Statevector(circuit)
    statevectors_list.append(statevector)
    #print(statevector)

    # Dummy target: we choose the all-|0> state (i.e. statevector with 1 in the first entry).
    # In practice, replace this with an objective that measures how "good" your feature vectors are.
    target = np.zeros(2**num_qubits, dtype=complex)
    target[0] = 1.0
    # Compute the L2 norm (Euclidean distance) between the statevector and the target.
    cost = np.linalg.norm(statevector - target)
    return cost



In [17]:
num_qubits = 4
big_statevectors = []
#for row in big_list:
  # Given list of 372 elements (example values from 1 to 372)
big_list = [
    [2, 2, 2, 1, 2, 2, 0, 0, 0, 0, 0, 2],
    [0, 0, 0, 1, 1, 1, 2, 0, 2, 0, 1, 0],
    [0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 0, 1],
    [2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [2, 2, 2, 1, 1, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 1, 1, 2, 1, 1, 1, 0, 2, 2, 0],
    [2, 0, 1, 2, 1, 1, 1, 1, 0, 2, 0, 2],
    [0, 0, 2, 0, 2, 2, 1, 1, 1, 0, 1, 1],
    [0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 2, 1],
    [2, 2, 0, 0, 1, 2, 1, 2, 2, 0, 1, 2],
    [2, 2, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0],
    [2, 0, 0, 0, 1, 1, 1, 2, 2, 0, 2, 1],
    [2, 2, 1, 0, 1, 2, 0, 1, 2, 2, 1, 2],
    [0, 2, 1, 0, 1, 2, 1, 1, 2, 2, 1, 0],
    [2, 0, 1, 0, 0, 2, 0, 2, 0, 0, 1, 0],
    [2, 1, 2, 0, 1, 0, 0, 2, 0, 2, 0, 2],
    [0, 1, 1, 0, 2, 1, 2, 1, 1, 1, 2, 1],
    [2, 2, 2, 0, 1, 2, 1, 1, 0, 0, 0, 0],
    [2, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 2],
    [0, 1, 1, 1, 2, 2, 0, 2, 2, 0, 1, 0]
]

for elements in big_list:
# Initialize a 20x20 matrix filled with -1
  matrix = np.full((4, 4), -1)

  # Fill the matrix with the given elements
  matrix.flat[:len(elements)] = elements

  # Convert to a list of lists
  grid = matrix.tolist()

  # Print the matrix (optional)
  for row in grid:
      print(row)

  # Initial guess: random parameters between 0 and pi.
  x0 = np.random.uniform(0, np.pi, size=2 * num_qubits)

  # Use a classical optimizer (COBYLA in this example) to minimize the cost function.
  result = minimize(cost_function, x0, method='COBYLA')

  print("Optimized parameters:")
  print(result.x)
  print("Final cost:", result.fun)
  print('Last Statevector:')
  print(statevectors_list[len(statevectors_list)-2])
  big_statevectors.append(statevectors_list[len(statevectors_list)-2])
  statevectors_list = []

  # ----- Extract Optimized Feature Vectors -----
  # With the optimized parameters, rebuild and simulate the circuit.
  opt_params1 = result.x[:num_qubits]
  opt_params2 = result.x[num_qubits:]
  opt_circuit = build_circuit(opt_params1, opt_params2, grid)

  # Here you can choose how to extract your feature vectors.
  # For instance, you might measure expectation values of certain observables
  # or simply use the statevector amplitudes (or a function of them) as features.
  qc_aer = transpile(opt_circuit, backend=AerSimulator())
  simulator_aer = AerSimulator()
  results = simulator_aer.run(qc_aer, shots=1024).result()
  counts=results.get_counts()
  final_state = qc_aer.remove_final_measurements()
  statevector = Statevector(qc_aer)
  print("Final statevector (can be processed to yield feature vectors):")
  print(final_state)



[2, 2, 2, 1]
[2, 2, 0, 0]
[0, 0, 0, 2]
[-1, -1, -1, -1]
Optimized parameters:
[ 7.82334105e-01  6.78732656e-01  3.60480404e+00  1.57081795e+00
 -8.95693609e-01 -1.54415076e-01  3.14158681e+00  1.31594046e-05]
Final cost: 0.81467460183705
Last Statevector:
Statevector([ 6.68152647e-01+1.74222462e-01j,
              8.07751473e-03-6.06524778e-01j,
              1.00432405e-02+2.39980226e-01j,
             -1.33769122e-05-2.22765472e-01j,
              1.56518690e-03-1.44598063e-01j,
             -1.39513812e-01+5.23850723e-02j,
             -2.30536376e-05-2.93529324e-02j,
             -5.18591690e-02-3.59843328e-02j,
              1.97572444e-06-6.42710103e-06j,
             -1.14943807e-05-2.74223586e-06j,
              9.85290150e-07+1.10767512e-05j,
              5.91892468e-06+3.58029501e-06j,
              2.71483181e-06+8.09714461e-06j,
              1.17654696e-05+6.77369741e-07j,
              9.70328665e-07-4.72036466e-06j,
             -5.33346342e-07-4.86440689e-06j],
       

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assume you have already imported Qiskit's Statevector class and your statevectors are available.
# from qiskit.quantum_info import Statevector
# For example, your statevectors list might look like:
# statevectors = [Statevector(...), Statevector(...), ...]  # 20 elements

# Your provided phenotype list for each patient:
phenotypes = ['LOW', 'LOW', 'HIGH', 'LOW', 'LOW', 'HIGH', 'HIGH', 'LOW', 'HIGH', 'LOW',
              'LOW', 'LOW', 'HIGH', 'HIGH', 'LOW', 'HIGH', 'HIGH', 'LOW', 'HIGH', 'LOW']

# --- Step 1: Convert statevectors into feature vectors ---
def statevector_to_features(sv):
    """
    Convert a Qiskit statevector into a feature vector by computing
    the squared magnitude (probability) of each amplitude.
    """
    # Assuming sv.data is a numpy array of complex amplitudes
    return np.abs(sv.data)**2

# Suppose your statevectors list is defined as "statevectors".
# Here we create a dummy list for illustration.
# In practice, replace this with your actual statevectors list.
from qiskit.quantum_info import Statevector
# For demonstration, we create 20 random statevectors for a 4-qubit circuit.
num_qubits = 4
statevectors = big_statevectors  # Replace with your actual list

# Convert each statevector to a feature vector (length 16)
features = np.array([statevector_to_features(sv) for sv in statevectors])  # shape: (20, 16)

# --- Step 2: Encode phenotype labels ---
# Convert 'LOW' to 0 and 'HIGH' to 1
labels = np.array([0 if pheno == 'LOW' else 1 for pheno in phenotypes])

# --- Step 3: Split into training (first 14) and test sets (last 6) ---
X_train = features[:14]  # shape: (14, 16)
y_train = labels[:14]    # shape: (14,)
X_test  = features[14:]  # shape: (6, 16)
y_test  = labels[14:]    # shape: (6,)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # shape: (14, 1)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)   # shape: (6, 1)

# --- Step 4: Define the Neural Network ---
class PhenotypeClassifier(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=32, output_dim=1):
        super(PhenotypeClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for binary classification

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

model = PhenotypeClassifier()

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary cross entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# --- Training Loop ---
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

print("Training complete.")

Epoch 10/100, Loss: 0.6718
Epoch 20/100, Loss: 0.6366
Epoch 30/100, Loss: 0.5814
Epoch 40/100, Loss: 0.5162
Epoch 50/100, Loss: 0.4615
Epoch 60/100, Loss: 0.4251
Epoch 70/100, Loss: 0.4012
Epoch 80/100, Loss: 0.3825
Epoch 90/100, Loss: 0.3652
Epoch 100/100, Loss: 0.3488
Training complete.


In [19]:
import random
import statistics
import pandas as pd

with open('liver.txt', 'r') as file:
    # Read the content of the file
    liver_data = file.read()

# with open('iron.txt', 'r') as file:
#     # Read the content of the file
#     iron_data = file.read()

# with open('bmi.txt', 'r') as file:
#     # Read the content of the file
#     bmi_data = file.read()


# Step 1: Extract SNPs and their weights into a dictionary
def extract_snps_and_weights(file_content):
    snp_dict = {}
    lines = file_content.split('\n')
    # print("LINES")
    # print(lines)
    for line in lines[1:]:  # Skip the first line (header)
        if line.startswith('rs'):
            parts = line.split()
            rs_id = parts[0]
            effect_weight = float(parts[5])
            snp_dict[rs_id] = effect_weight
    return snp_dict

# Step 2: Generate data for 20 people
def generate_person_data(snp_dict):
    people_data = []
    for num in range(1):
        person_data = {}
        for rs_id in snp_dict:
            # Randomly assign 0, 1, or 2 effective alleles for each SNP
            person_data[rs_id] = random.randint(0, 2)
        people_data.append(person_data)
    return people_data


def generate_final_csv(generated_csv, snp_dict):

    df = pd.DataFrame(generated_csv)
    PGS_list = []
    for person in generated_csv:
        pgs_score = 0
        for snp in person:
            pgs_score+=(snp_dict[snp] * person[snp])
        PGS_list.append(pgs_score)

    average_PGS = statistics.mean(PGS_list)

    PGS_high_low = []

    for index, person in enumerate(generated_csv):
        if PGS_list[index] < average_PGS:
            PGS_high_low.append("LOW")
        else:
            PGS_high_low.append("HIGH")

    df["phenotype"] = PGS_high_low
    return df

processed_liver = extract_snps_and_weights(liver_data)
#processed_bmi = extract_snps_and_weights(bmi_data)
#processed_iron = extract_snps_and_weights(iron_data)

generated_liver = generate_person_data(processed_liver)
#generated_bmi = generate_person_data(processed_bmi)
#generated_iron = generate_person_data(processed_iron)

genotype_array = [generated_liver[0][rs] for rs in processed_liver]
print(genotype_array)

[0, 2, 2, 2, 2, 0, 1, 0, 1, 0, 0, 0]


In [20]:
elements = genotype_array
#Initialize a 20x20 matrix filled with -1
matrix = np.full((4, 4), -1)

# Fill the matrix with the given elements
matrix.flat[:len(elements)] = elements

# Convert to a list of lists
grid = matrix.tolist()

# Initial guess: random parameters between 0 and pi.
x0 = np.random.uniform(0, np.pi, size=2 * num_qubits)

# Use a classical optimizer (COBYLA in this example) to minimize the cost function.
result = minimize(cost_function, x0, method='COBYLA')

print("Optimized parameters:")
print(result.x)
print("Final cost:", result.fun)
print('Last Statevector:')
print(statevectors_list[len(statevectors_list)-2])
big_statevectors.append(statevectors_list[len(statevectors_list)-2])
statevectors_list = []

# ----- Extract Optimized Feature Vectors -----
# With the optimized parameters, rebuild and simulate the circuit.
opt_params1 = result.x[:num_qubits]
opt_params2 = result.x[num_qubits:]
opt_circuit = build_circuit(opt_params1, opt_params2, grid)

# Here you can choose how to extract your feature vectors.
# For instance, you might measure expectation values of certain observables
# or simply use the statevector amplitudes (or a function of them) as features.
qc_aer = transpile(opt_circuit, backend=AerSimulator())
simulator_aer = AerSimulator()
results = simulator_aer.run(qc_aer, shots=1024).result()
counts=results.get_counts()
final_state = qc_aer.remove_final_measurements()
statevector = Statevector(qc_aer)
print("Final statevector (can be processed to yield feature vectors):")
print(final_state)

Optimized parameters:
[-1.31398198e-05  3.78509205e+00  6.43413066e-01  3.14156115e+00
  1.57081690e+00  1.57085152e+00  1.57082225e+00  1.57076454e+00]
Final cost: 0.93541434773783
Last Statevector:
Statevector([ 5.62500000e-01-5.84561868e-01j,
              3.98746221e-11+1.01375109e-07j,
             -4.31500751e-11-1.89187453e-06j,
             -1.69569832e-06-3.65137514e-06j,
              4.39496701e-07-4.56687263e-07j,
             -1.19976951e-05+1.29915685e-01j,
              8.01978259e-06+7.09322616e-06j,
              3.45415946e-06-2.98441487e-06j,
              8.90769931e-07+4.47020629e-06j,
              2.97187156e-06-2.35789628e-06j,
              4.87293278e-08+5.08671747e-08j,
              3.87436094e-06-5.62917834e-01j,
             -3.34121067e-11-2.56012991e-06j,
              1.54182454e-05-8.00121637e-06j,
              6.24999999e-02+6.49642199e-02j,
              4.53994508e-11-4.39723418e-07j],
            dims=(2, 2, 2, 2))
Final statevector (can be proces

In [21]:
print(big_statevectors[0])

Statevector([ 6.68152647e-01+1.74222462e-01j,
              8.07751473e-03-6.06524778e-01j,
              1.00432405e-02+2.39980226e-01j,
             -1.33769122e-05-2.22765472e-01j,
              1.56518690e-03-1.44598063e-01j,
             -1.39513812e-01+5.23850723e-02j,
             -2.30536376e-05-2.93529324e-02j,
             -5.18591690e-02-3.59843328e-02j,
              1.97572444e-06-6.42710103e-06j,
             -1.14943807e-05-2.74223586e-06j,
              9.85290150e-07+1.10767512e-05j,
              5.91892468e-06+3.58029501e-06j,
              2.71483181e-06+8.09714461e-06j,
              1.17654696e-05+6.77369741e-07j,
              9.70328665e-07-4.72036466e-06j,
             -5.33346342e-07-4.86440689e-06j],
            dims=(2, 2, 2, 2))


In [22]:
X_test = big_statevectors[0]
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
test_outputs = model(X_test_tensor)
predicted = (test_outputs > 0.5).float()
if predicted[0].item == 1.0:
  label = 'HIGH'
else:
  label = 'LOW'
print(label)

LOW
